In [125]:
import pandas as pd

# import packages
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error as mse
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, train_test_split, KFold 
from sklearn.model_selection import cross_val_score
from sklearn import linear_model, metrics
from sklearn.metrics import mean_squared_error, r2_score
df = pd.read_csv('../../../data/kc_house_data.csv')

In [126]:
df

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,263000018,5/21/2014,360000.0,3,2.50,1530,1131,3.0,0.0,0.0,...,8,1530,0.0,2009,0.0,98103,47.6993,-122.346,1530,1509
21593,6600060120,2/23/2015,400000.0,4,2.50,2310,5813,2.0,0.0,0.0,...,8,2310,0.0,2014,0.0,98146,47.5107,-122.362,1830,7200
21594,1523300141,6/23/2014,402101.0,2,0.75,1020,1350,2.0,0.0,0.0,...,7,1020,0.0,2009,0.0,98144,47.5944,-122.299,1020,2007
21595,291310100,1/16/2015,400000.0,3,2.50,1600,2388,2.0,NaN,0.0,...,8,1600,0.0,2004,0.0,98027,47.5345,-122.069,1410,1287


In [127]:
df = df.replace('?', np.nan)

In [128]:
df.sqft_basement = df.sqft_basement.replace('?', np.nan)

In [129]:
df.sqft_basement = df.sqft_basement.astype(float)

In [130]:
df = df.replace(np.nan, '0')

In [131]:
X = df[['sqft_living', 'sqft_lot', 'sqft_living15', 'sqft_lot15', 'sqft_above', 'bathrooms', 'bedrooms',  'lat', 'long', 'zipcode', 'grade', 'yr_built', 'yr_renovated', 'view', 'condition', 'grade', 'sqft_basement']]
y = df['price']

In [132]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=42)

In [133]:
lr = LinearRegression()

In [134]:
X_test.isna().sum()

sqft_living      0
sqft_lot         0
sqft_living15    0
sqft_lot15       0
sqft_above       0
bathrooms        0
bedrooms         0
lat              0
long             0
zipcode          0
grade            0
yr_built         0
yr_renovated     0
view             0
condition        0
grade            0
sqft_basement    0
dtype: int64

In [135]:
lr.fit(X_train, y_train)

LinearRegression()

In [136]:
lr.score(X_test, y_test)

0.6749801766510155

In [137]:
print('Intercept: {}'.format(lr.intercept_))
print('Slope: {}'.format(lr.coef_))

Intercept: 5847708.651645162
Slope: [ 8.37235990e+01  6.41764221e-02  2.36456866e+01 -4.23743578e-01
  9.44716764e+01  4.49044616e+04 -3.94017706e+04  5.83078694e+05
 -2.14040993e+05 -5.63097871e+02  4.96449707e+04 -2.64690829e+03
  2.71158200e+01  7.81108175e+04  2.96156529e+04  4.96449707e+04
  6.27659350e+01]


In [138]:
# Make predictions for train and test set
train_preds1 = lr.predict(X_train)
test_preds1 = lr.predict(X_test)

In [139]:
print('Training RMSE: {}'.format(np.sqrt(mse(y_train, train_preds1))))
print('Testing RMSE: {}'.format(np.sqrt(mse(y_test, test_preds1))))

Training RMSE: 205095.98689943543
Testing RMSE: 211169.66086557135


In [140]:
X_train = X_train.loc[y_train.index]

In [141]:
X_train

,sqft_living,sqft_lot,sqft_living15,sqft_lot15,sqft_above,bathrooms,bedrooms,lat,long,zipcode,grade,yr_built,yr_renovated,view,condition,grade,sqft_basement
6405,1880,4499,2130,5114,1880,2.50,3,47.5664,-121.999,98029,8,1993,0,0,3,8,0
937,2020,6564,1710,5151,1310,2.50,3,47.3545,-122.158,98042,7,1994,0,0,3,7,710
19076,4720,493534,2160,219542,3960,4.00,5,47.4536,-122.009,98027,9,1975,0,0,5,9,760
15201,1430,3880,1430,3880,1430,2.00,2,47.6844,-122.392,98117,7,1949,0,0,4,7,0
13083,2270,32112,2310,41606,1740,2.25,3,47.3451,-122.094,98042,8,1980,0,0,4,8,530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11964,2230,5800,2230,6088,2230,2.50,3,47.5308,-121.847,98065,7,2004,0,0,3,7,0
21575,2770,3852,1810,5641,2770,2.75,4,47.5001,-122.232,98178,8,2014,0,0,3,8,0
5390,1530,9000,1520,8500,1530,1.50,4,47.6492,-121.908,98014,6,1976,0,0,4,6,0
860,380,15000,1170,15000,380,0.75,1,47.4810,-122.323,98168,5,1963,0,0,3,5,0


In [142]:
X_train.sqft_basement = X_train.sqft_basement.replace('?', np.nan)

In [143]:
X_train.sqft_basement = X_train.sqft_basement.astype(float)

In [144]:
X_train['target'] = y_train

In [145]:
X_train.dtypes

sqft_living        int64
sqft_lot           int64
sqft_living15      int64
sqft_lot15         int64
sqft_above         int64
bathrooms        float64
bedrooms           int64
lat              float64
long             float64
zipcode            int64
grade              int64
yr_built           int64
yr_renovated      object
view              object
condition          int64
grade              int64
sqft_basement    float64
target           float64
dtype: object

In [146]:
X_train[(np.abs(stats.zscore(X_train[['sqft_living', 'sqft_lot', 'sqft_living15', 'sqft_lot15', 'sqft_above', 'bathrooms', 'bedrooms', 'target']])) < 3).all(axis=1)].drop_duplicates()

,sqft_living,sqft_lot,sqft_living15,sqft_lot15,sqft_above,bathrooms,bedrooms,lat,long,zipcode,grade,yr_built,yr_renovated,view,condition,grade,sqft_basement,target
6405,1880,4499,2130,5114,1880,2.50,3,47.5664,-121.999,98029,8,1993,0,0,3,8,0.0,529000.0
937,2020,6564,1710,5151,1310,2.50,3,47.3545,-122.158,98042,7,1994,0,0,3,7,710.0,253000.0
15201,1430,3880,1430,3880,1430,2.00,2,47.6844,-122.392,98117,7,1949,0,0,4,7,0.0,545000.0
13083,2270,32112,2310,41606,1740,2.25,3,47.3451,-122.094,98042,8,1980,0,0,4,8,530.0,390000.0
16852,1048,4800,950,4800,1048,1.50,1,47.4759,-122.212,98055,7,1942,0,0,3,7,0.0,199950.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11964,2230,5800,2230,6088,2230,2.50,3,47.5308,-121.847,98065,7,2004,0,0,3,7,0.0,440000.0
21575,2770,3852,1810,5641,2770,2.75,4,47.5001,-122.232,98178,8,2014,0,0,3,8,0.0,572000.0
5390,1530,9000,1520,8500,1530,1.50,4,47.6492,-121.908,98014,6,1976,0,0,4,6,0.0,299800.0
860,380,15000,1170,15000,380,0.75,1,47.4810,-122.323,98168,5,1963,0,0,3,5,0.0,245000.0


In [174]:
X = df[['sqft_living', 'sqft_lot', 'sqft_living15', 'sqft_lot15', 'sqft_above', 'bathrooms', 'bedrooms',  'lat', 'long', 'zipcode', 'grade', 'yr_built', 'condition', 'grade']]
y = df['price']

In [175]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=42)

In [176]:
X_train['target'] = y_train
X_train_no = X_train[(np.abs(stats.zscore(X_train[['sqft_living', 'sqft_lot', 'sqft_living15', 'sqft_lot15', 'sqft_above', 'bathrooms', 'bedrooms', 'target']])) < 3).all(axis=1)].drop_duplicates()
y_train_no = X_train_no['target']
X_train_no = X_train_no.drop('target', axis = 1)


<ipython-input-176-5e35db3e1a1c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['target'] = y_train


In [177]:
lr = LinearRegression()

In [178]:
lr.fit(X_train_no, y_train_no)

LinearRegression()

In [179]:
lr.score(X_test, y_test)

0.5974311169818393

In [180]:
print('Intercept: {}'.format(lr.intercept_))
print('Slope: {}'.format(lr.coef_))

Intercept: -10841613.350749869
Slope: [ 9.60608408e+01  2.28153443e-01  5.97762211e+01 -1.34728661e+00
  6.09964581e+00  3.97595843e+04 -2.18642486e+04  5.43852284e+05
 -1.16337294e+05 -2.56699843e+02  4.64735343e+04 -2.37251020e+03
  2.59826344e+04  4.64735343e+04]


In [181]:
# Make predictions for train and test set
train_preds1 = lr.predict(X_train_no)
test_preds1 = lr.predict(X_test)

In [182]:
print('Training RMSE: {}'.format(np.sqrt(mse(y_train_no, train_preds1))))
print('Testing RMSE: {}'.format(np.sqrt(mse(y_test, test_preds1))))

Training RMSE: 142027.29420188637
Testing RMSE: 235015.60417597834


In [161]:
# Isolate the features for the first model
model_1_train = X_train[['sqft_living', 'sqft_lot', 'sqft_living15', 'sqft_lot15', 'sqft_above', 'bathrooms', 'bedrooms',  'lat', 'long', 'zipcode', 'grade', 'yr_built', 'condition', 'grade']]

model_1_test = X_test[['sqft_living', 'sqft_lot', 'sqft_living15', 'sqft_lot15', 'sqft_above', 'bathrooms', 'bedrooms',  'lat', 'long', 'zipcode', 'grade', 'yr_built', 'condition', 'grade']]

# Initialize a model
model1 = LinearRegression()

# Fit the model
model1.fit(model_1_train, y_train)

# Generate training and test predictions
train_preds2 = model1.predict(model_1_train)
test_preds2 = model1.predict(model_1_test)

# Calculate training and test error
train_error = mean_squared_error(y_train, train_preds)
test_error = mean_squared_error(y_test, test_preds)

# What is the difference in training and test error?
difference = test_error - train_error

# Print results
print('Train error:', train_error)
print('Test error:', test_error)
print('Difference:', difference)
print('Training RMSE: {}'.format(np.sqrt(mse(y_train, train_preds2))))
print('Testing RMSE: {}'.format(np.sqrt(mse(y_test, test_preds2))))

Train error: 45122338460.7855
Test error: 48007921586.20227
Difference: 2885583125.416771
Training RMSE: 212420.19315683126
Testing RMSE: 219107.1007206345


In [164]:
def model(X_train, X_test, y_train, y_test, features):
    # Isolate the features for the first model
    model_train = X_train[features]
    model_test = X_test[features]
    
    # Initialize a model
    model = LinearRegression()
    
    # Fit the model
    model.fit(model_train, y_train)
    
    # Generate training and test predictions
    train_preds3 = model.predict(model_train)
    test_preds3 = model.predict(model_test)
    
    # Calculate training and test error
    train_error = mean_squared_error(y_train, train_preds)
    test_error = mean_squared_error(y_test, test_preds)
    
    # What is the difference in training and test error?
    difference = test_error - train_error
    print('Training RMSE: {}'.format(np.sqrt(mse(y_train, train_preds3))))
    print('Testing RMSE: {}'.format(np.sqrt(mse(y_test, test_preds3))))
    # return results
    return train_error, test_error, difference

second_model_features = ['sqft_living', 'sqft_lot', 'sqft_living15', 'sqft_lot15', 'sqft_above', 'bathrooms', 'bedrooms']
model_2_results = model(X_train, X_test, y_train, y_test, second_model_features)
model_2_results


Training RMSE: 253953.25497710324
Testing RMSE: 258453.08877703606


(45122338460.7855, 48007921586.20227, 2885583125.416771)

In [186]:
# Isolate the features for the first model
model_3_train = X_train_no[['sqft_living', 'sqft_lot', 'sqft_living15', 'sqft_lot15', 'sqft_above', 'bathrooms', 'bedrooms', 'grade', 'yr_built', 'condition', 'grade']]

model_3_test = X_test[['sqft_living', 'sqft_lot', 'sqft_living15', 'sqft_lot15', 'sqft_above', 'bathrooms', 'bedrooms', 'grade', 'yr_built', 'condition', 'grade']]

# Initialize a model
model3 = LinearRegression()

# Fit the model
model3.fit(model_3_train, y_train_no)

# Generate training and test predictions
train_preds3 = model3.predict(model_3_train)
test_preds3 = model3.predict(model_3_test)

# Calculate training and test error
train_error = mean_squared_error(y_train_no, train_preds3)
test_error = mean_squared_error(y_test, test_preds3)

# What is the difference in training and test error?
difference = test_error - train_error

# Print results
print('Train error:', train_error)
print('Test error:', test_error)
print('Difference:', difference)
print('Training RMSE: {}'.format(np.sqrt(mse(y_train_no, train_preds3))))
print('Testing RMSE: {}'.format(np.sqrt(mse(y_test, test_preds3))))

Train error: 25177423529.906178
Test error: 62572899191.83046
Difference: 37395475661.924286
Training RMSE: 158673.95353335777
Testing RMSE: 250145.7558940996


In [187]:
# Isolate the features for the first model
model_4_train = X_train[['sqft_living', 'sqft_lot', 'sqft_living15', 'sqft_lot15', 'sqft_above', 'bathrooms', 'bedrooms', 'grade', 'yr_built', 'condition', 'grade']]

model_4_test = X_test[['sqft_living', 'sqft_lot', 'sqft_living15', 'sqft_lot15', 'sqft_above', 'bathrooms', 'bedrooms', 'grade', 'yr_built', 'condition', 'grade']]

# Initialize a model
model4 = LinearRegression()

# Fit the model
model4.fit(model_4_train, y_train)

# Generate training and test predictions
train_preds4 = model4.predict(model_4_train)
test_preds4 = model4.predict(model_4_test)

# Calculate training and test error
train_error = mean_squared_error(y_train, train_preds4)
test_error = mean_squared_error(y_test, test_preds4)

# What is the difference in training and test error?
difference = test_error - train_error

# Print results
print('Train error:', train_error)
print('Test error:', test_error)
print('Difference:', difference)
print('Training RMSE: {}'.format(np.sqrt(mse(y_train, train_preds4))))
print('Testing RMSE: {}'.format(np.sqrt(mse(y_test, test_preds4))))

Train error: 50320262100.56606
Test error: 54725802155.0114
Difference: 4405540054.445335
Training RMSE: 224321.78249239654
Testing RMSE: 233935.46579134042
